<a href="https://colab.research.google.com/github/AkashKoley012/Pytorch-/blob/main/Qustion%20Answer(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [16]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?', '')
  text = text.replace('"', '')
  text = text.replace("'", "")
  return text.split()

In [17]:
tokenize("Who wrote 'To Kill a Mockingbird'?")

['who', 'wrote', 'to', 'kill', 'a', 'mockingbird']

In [4]:
# vocab
vocab = {'<UNK>':0}

In [10]:
def build_vocab(row):
  question = tokenize(row['question'])
  answer = tokenize(row['answer'])

  merged = question + answer

  for token in merged:
    if token not in vocab:
      vocab[token] = len(vocab)

In [11]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [12]:
len(vocab)

324

In [20]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return torch.tensor(indexed_text)

In [18]:
text_to_indices('Who wrote "To Kill a Mockingbird"?', vocab)

[10, 11, 12, 13, 14, 15]

In [19]:
# Dataset & DataLoader
import torch
from torch.utils.data import Dataset, DataLoader

In [21]:
class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return numerical_question, numerical_answer

In [22]:
dataset = QADataset(df, vocab)

In [23]:
dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [24]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [25]:
for question, answer in dataloader:
  print(question, answer)

tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([[166]])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([[121]])
tensor([[ 10,  11, 157, 158, 159]]) tensor([[160]])
tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]]) tensor([[23]])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([[316]])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([[9]])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([[113]])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([[36]])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([[268]])
tensor([[ 10,   2,  62,  63,   3, 283,   5, 284]]) tensor([[285]])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([[205]])
tensor([[  1,   2,   3,  37, 133,   5,  26]]) tensor([[134]])
tensor([[ 1,  2,  3, 37, 38, 39, 40]]) tensor([[41]])
tensor([[ 10,  75, 111]]) tensor([[112]])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([[321]])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([[128]])
tensor([[  

In [41]:
import torch.nn as nn

class SimpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, x):
    x = self.embedding(x)
    hidden, final = self.rnn(x)
    return self.fc(final.squeeze(0))

In [26]:
dataset[15][0]

tensor([ 1,  2,  3, 69,  5,  3, 70, 71])

In [29]:
x = nn.Embedding(324, embedding_dim=50)

In [33]:
a = x(dataset[15][0])

In [34]:
y = nn.RNN(50, 64)

In [37]:
y(a) # it's give 2 output all outputs like o1, o2 ... and default second result y(a)[last] that's why we can't use Sequence in RNN

(tensor([[ 0.2894, -0.5706, -0.7059,  0.7176,  0.6470,  0.8331, -0.5472, -0.4395,
          -0.0063,  0.1847,  0.3144, -0.2123,  0.0451, -0.6317,  0.2931,  0.7366,
           0.2810,  0.6035,  0.1643, -0.3817, -0.0220, -0.7655,  0.2141,  0.3367,
          -0.5148, -0.1514, -0.4367,  0.0986, -0.0858, -0.0913,  0.5043,  0.7579,
          -0.2508,  0.8013, -0.0048,  0.4639,  0.5712,  0.5513, -0.8420, -0.5529,
           0.7679,  0.2198,  0.6275, -0.5613,  0.3327, -0.3699,  0.3364, -0.2330,
           0.7900,  0.1709, -0.3731, -0.0271,  0.0997,  0.8538,  0.1265,  0.9256,
          -0.0664, -0.3627,  0.5456, -0.1017,  0.5014,  0.2020,  0.4647,  0.6648],
         [-0.6779,  0.7503, -0.5404, -0.3745, -0.4546,  0.2106,  0.4981, -0.0643,
          -0.7985, -0.6211, -0.3829, -0.6785,  0.4192, -0.8547,  0.4916, -0.1907,
           0.4268,  0.5849,  0.7266,  0.3416,  0.8629,  0.1159, -0.8068, -0.6726,
           0.3768,  0.0517, -0.5337, -0.1140, -0.6101,  0.2802,  0.8112, -0.5381,
           0.00

In [39]:
lr =0.001
epochs = 20

In [42]:
model = SimpleRNN(len(vocab))

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [44]:
# training loop
for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    y_hat = model(question)

    loss = loss_fn(y_hat, answer[0])

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f'epoch: {epoch+1} loss: {total_loss:4f}')

epoch: 1 loss: 527.268655
epoch: 2 loss: 462.380653
epoch: 3 loss: 387.529388
epoch: 4 loss: 321.539518
epoch: 5 loss: 267.510517
epoch: 6 loss: 218.931633
epoch: 7 loss: 175.167384
epoch: 8 loss: 136.673164
epoch: 9 loss: 105.979186
epoch: 10 loss: 81.735008
epoch: 11 loss: 63.305677
epoch: 12 loss: 49.511766
epoch: 13 loss: 39.193814
epoch: 14 loss: 31.625880
epoch: 15 loss: 25.706715
epoch: 16 loss: 21.307259
epoch: 17 loss: 17.655009
epoch: 18 loss: 14.933520
epoch: 19 loss: 12.855155
epoch: 20 loss: 11.059533


In [45]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [47]:
predict(model, "What is the largest planet in our solar system?")

jupiter


<ipython-input-45-dd56cb346c6b>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)


In [48]:
list(vocab.keys())[7]

'paris'